# Deriving a Point Spread Function an a Crowded Field
### ...following Appendix III of Peter Stetson's *User's Manual for DAOPHOT II*
### Using `pydaophot` form `astwro` python package

All *italic* text here have been taken from Ststson's manual.

The only input file for this procedure is a FITS file containing reference frame image. Here we use sample FITS form astwro package (NGC6871 I filter 20s frame). Below we get filepath for this image, as well as instance `DPRunner` and `ASRunner` objects - wrappers around `(s)daophot` and `(s)allstar` respectively.

One can also provide `daophot.opt`, `photo.opt` and `allstar.opt` in apropiriete constructors. Here default, build in, `opt` files are used.

In [1]:
from astwro.sampledata import fits_image
frame = fits_image()

In [2]:
from astwro.pydaophot import daophot, allstar
dp = daophot(image_file=frame)
al = allstar(dir=dp.dir)

Daophot got FITS file in construction, which will be automatically **ATTACH**ed. Wrapper created temporary working directory (one can provide it's own via `dir` parameter).

Allstar got working dir from daophot to share.

#### *(1) Run FIND on your frame*

In [3]:
dp.FInd(frames_av=1, frames_sum=1)

FInd parameters `Number of frames averaged, summed` are defaulted to `1,1`, below are provided for clearity.

#### *(2) Run PHOTOMETRY on your frame*

In [4]:
dp.PHotometry()

#### *(3) SORT the output from PHOTOMETRY*
*in order of increasing apparent magnitude  decreasing
stellar brightness   with the renumbering feature.  This step is optional  but it can be more convenient than not.*

`SORT` command id not implemented (yet) in `pydaohot`, so we skip it.

#### *(4) PICK to generate a set of likely PSF stars*  
*How many stars you want to use is a function of the degree of variation you exp ect  and the frequency with which stars are contaminated by cosmic rays or neighbor stars. [...]*

In [5]:
dp.PIck(faintest_mag=20, number_of_stars_to_pick=40)

** No daophot command was executed yet! **

All daophot commands were stacked up in DPRunner command buffer, and not executed yet! Let it run, it can take a while.

In [6]:
dp.run()

If no error reported, symlink to image file (renamed to `i.fits`), and all daophot otput files (`i.*`) are  in the working directory of runner:

In [7]:
ls $dp.dir

allstar.opt  i.ap         i.fits@      photo.opt
daophot.opt  i.coo        i.lst


One may examine and improve `i.lst` list of PSF stars. Or use `astwro.tools.gapick.py` to obtain list of PSF stars optimized by genetic algorithm.

Also, we chan check results of executed command, which was preinted to sceen by daophot. For each command `XXX` there is `XXX_result` memeber of runner.

In [8]:
print ('Star mode: {}. FIND found {} stars. {} was picked by PICK'.format(
        dp.FInd_result.sky,
        dp.FInd_result.stars,
        dp.PIck_result.stars
    ))

Star mode: 12.665. FIND found 4166 stars. 40 was picked by PICK


####  *(5) Run PSF *
*tell it the name of your complete  (sorted  renumbered)  aperture photometry  file,  the name of the  file with the list of PSF stars,  and the name of the disk  file you want the point spread function stored in (the default should be  fine) [...]*

*If the frame is crowded  it is probably worth your while to generate the  first PSF with the  "VARIABLE PSF"  option set to -1 ---  pure analytic PSF.  That way,  the companions will not generate ghosts in the model PSF that will come back to haunt you later.  You should also have specified a reasonably generous  fitting radius ---  these stars have been preselected to be as isolated as possible  and you want the best  fits you can get.  But remember to avoid letting neighbor stars intrude within one  fitting radius of the center of any PSF star.*


For illustration we will simulate temporary change of `VARIABLE PSF` option, but giving it default value `2` which works good on smaple image. Chages made by `OPTION` command are valid only for next run.

In [9]:
dp.OPtion('VARIABLE PSF', 2)
dp.PSf()
dp.run()

#### *(6) Run GROUP and NSTAR or ALLSTAR on your NEI  file*
*If your PSF stars have many neighbors this may take some minutes of real time.  Please be patient  or submit it as a batch job and perform steps       on your next frame while you wait.*

We use `allstar`. (`GROUP` and `NSTAR` command are not implemented in current version of `pydaophot`). We use prepared above `ASRunner`, `al` opereating on the same workdir. By default `ASRunner` operates on all stars list `i.coo`, so we have to provide it with NEI file (using predefined constants `fname` with filenames scheme used in working dir).

In [10]:
from astwro.pydaophot import fname
al.input_file = fname.NEI_FILE
al.run()

All `result` objects, has `get_buffer()` method, usefull to check unparsed `daophot/allstar` output:

In [11]:
print (al.result.get_buffer())

     i...                                    


                                      Picture size:   1250  1150


                 File with the PSF (default i.psf):                          Input file (default i.ap):                   File for results (default i.als):             Name for subtracted image (default is): 
     916 stars.  <<


 I = iteration number

 R = number of stars that remain

 D = number of stars that disappeared

 C = number of stars that converged



      I       R       D       C
      1     916       0       0  <<
      2     916       0       0  <<
      3     916       0       0  <<
      4     726       0     190  <<
      5     385       0     531  <<
      6     211       0     705  <<
      7     108       0     808  <<
      8      67       0     849  <<
      9      41       0     875  <<
     10       0       0     916

     Finished i                                       


 Good bye.




#### *(8) EXIT from DAOPHOT and send this new picture to the image display * 
*Examine each of the PSF stars and its environs.  Have all of the PSF stars subtracted out more or less cleanly,  or should some of them be rejected from further use as PSF stars?  (If so  use a text editor to delete these stars from the  LST  file.)   Have the neighbors mostly disappeared,  or have they left behind big zits?  Have you uncovered any faint companions that FIND missed?[...]* 

The path to substracted file for inspection can be obtained by `file_from_working_dir` method.

In [13]:
sub_img = dp.file_from_working_dir(fname.SUBTRACTED_IMAGE_FILE)

In [16]:
# One can run ds9 directly from notebook:
# !ds9 $sub_img
print sub_img

/var/folders/0d/0cwxd3cx34l7cqbb_74zczqw0000gp/T/pydaophot_tmp0JlVcW/i.sub.fits


#### *(9) Back in DAOPHOT II  ATTACH the original picture and run SUBSTAR*
*specifying the  file created in step (6) or in step (8f) as the stars to subtract,  and the stars in the  LST  file as the stars to keep.  You have now created a new picture which has the PSF stars still in it  but from which the known neighbors of these PSF stars have been mostly removed*

In [ ]:
do.SUbstract(substract=fname.ALS_FILE, keep=fname.LST_FILE)

IMPLEMENT SUB jak wyżej!

'/var/folders/0d/0cwxd3cx34l7cqbb_74zczqw0000gp/T/pydaophot_tmpsbcLfR/i.sub.fits'

dp.file_from_working_dir(fname.SUBTRACTED_IMAGE_FILE)

In [19]:
ls $al.dir

allstar.opt  i.als        i.coo        i.fits@      i.nei        is.fits
daophot.opt  i.ap         i.err        i.lst        i.psf        photo.opt
